In [1]:
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import itertools
import csv , codecs
import glob
from PIL import Image
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn
from sklearn import decomposition
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [2]:
data_dir = "./EEG_data/"

엑셀 파일에 채널이 35개가 있으나 채널 32개만 두피에 연결된 채널만을 말합니다. 

35개의 채널 중에는 EOG1, EOG2, ECG 채널이 포함되어 있습니다.
각각 눈 움직임 (+), (-) 그리고 심박동을 측정하는 전극입니다. 
마지막 33, 34, 35 채널을 제외하고 진행합니다.

Abs 파워는 delta ~ gamma 로 나눠진 주파수 밴드 중 각 뇌파가 갖고있는 파워의 절대값입니다.
Alpha파를 예로 들자면, 8~13Hz 에 포함된 파워들의 모든 합이 alpha파의 abs 파워가 됩니다.

In [ ]:
Relative 파워는 각 뇌파가 가지고 있는 뇌파가 전체로 봤을 때 몇 %를 차지하는가를 보는것입니다.
Alpha파의 Relative 파워는 delta~gamma 까지의 주파수 범위인 0.1 ~ 50 Hz 에 포함된 파워들을 분모로 두고,
절대 파워를 분자로 둬서 나온 비율입니다. 

## CSV 파일 생성 함수

In [3]:
def EEG_data_csv(data_dir):
    
    # Kinds of Power 입력
    file_list = os.listdir(data_dir)
    file_list.sort()
       
    global kinds_of_power
    kinds_of_power = str(input("Relative, Abs :"))
    
    excel_dir = "./EEG_data" + '/' + kinds_of_power
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    # EEG_csv_data 디렉토리 생성
    csv_dir = "./EEG_csv_data"
    if not os.path.isdir(csv_dir):
        os.mkdir(csv_dir)
    
    Abs_dir ="./EEG_csv_data/Abs"
    Relative_dir = "./EEG_csv_data/Relative"
    
    if not os.path.isdir(Abs_dir):
        os.mkdir(Abs_dir)
    if not os.path.isdir(Relative_dir):
        os.mkdir(Relative_dir)
    
    book = Workbook()
    sheet = book.active
    
    # x축,y축 생성
    for i in range (1,21) :
        if i < 10 :
            sheet['A'+str(i+1)] = 'S0'+str(i)
        if i == 10 :
            sheet['A'+str(i+1)] = 'S'+str(10)
        if i > 10 :
            sheet['A'+str(i+1)] = 'S'+str(i)
            
    for j in range (0,12):
        for i in range(1,33):
            sheet.cell(row=1, column=160*j+i+1).value = 'ACh' + str(i)
            sheet.cell(row=1, column=160*j+i+33).value = 'BCh' + str(i)
            sheet.cell(row=1, column=160*j+i+65).value = 'DCh' + str(i)
            sheet.cell(row=1, column=160*j+i+97).value = 'GCh' + str(i)
            sheet.cell(row=1, column=160*j+i+129).value = 'TCh' + str(i)

    # 파일을 순차적으로 열어서 셀 영역을 복사
    # j: 엑셀 파일의 수, k: 각 엑셀 파일 당 시트의 수, n: task의 수, l: subject의 수, m: ch1-ch32

    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=l,column=160*k+m+32*j).value = source.cell(row=l,column=m).value

    # EEG_csv_data 디렉토리에 csv 파일로 저장
    if kinds_of_power == "Abs":
        with open(Abs_dir + str('/') +'Abs_Merged.csv', 'w', newline="") as f:
            c = csv.writer(f)
            for r in sheet.rows:
                c.writerow([cell.value for cell in r])
    elif kinds_of_power == "Relative":
        with open(Relative_dir + str('/') +'Relative_Merged.csv', 'w', newline="") as f:
            c = csv.writer(f)
            for r in sheet.rows:
                c.writerow([cell.value for cell in r])

## CSV 파일에서 Pandas dataframe으로 가져오는 함수

In [4]:
EEG_data_csv(data_dir)


### task 0 1 2 : 1-back,  2-back,  rest

### eye 0  1    : close eye,     open eye

### respiratory 0 1 2 : mouse, nose, O2 



    df["target"][0*subject:1*subject] = "010"
    df["target"][1*subject:2*subject] = "011"
    df["target"][2*subject:3*subject] = "012"

    df["target"][3*subject:4*subject] = "110"
    df["target"][4*subject:5*subject] = "111"
    df["target"][5*subject:6*subject] = "112"

    df["target"][6*subject:7*subject] = "200"
    df["target"][7*subject:8*subject] = "201"
    df["target"][8*subject:9*subject] = "202"

    df["target"][9*subject:10*subject] = "210"
    df["target"][10*subject:11*subject] = "211"
    df["target"][11*subject:12*subject] = "212"

In [5]:
def dataset_csv_3():
    normalize_power = str(input("Relative, Abs :"))
    excel_dir = "./EEG_data" + '/' + normalize_power
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    subject = int(input("subject :"))
    book = Workbook()
    sheet = book.active
    sheet.cell(row=1, column=161).value = 'target'
 
    # x축 생성   
    for i in range(1,33):
        sheet.cell(row=1, column=i).value = 'ACh' + str(i)
        sheet.cell(row=1, column=i+32).value = 'BCh' + str(i)
        sheet.cell(row=1, column=i+64).value = 'DCh' + str(i)
        sheet.cell(row=1, column=i+96).value = 'GCh' + str(i)
        sheet.cell(row=1, column=i+128).value = 'TCh' + str(i)
        
    # 파일을 순차적으로 열어서 셀 영역을 복사
    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=20*k+l,column=32*j+m-1).value = source.cell(row=l,column=m).value
        

    with open('./dataset_eeg_3.csv', 'w', newline="") as f:
        c = csv.writer(f)
        for r in sheet.rows:
            c.writerow([cell.value for cell in r])
            
    df = pd.read_csv('./dataset_eeg_3.csv')
    df[df =='                      NaN'] =np.nan
    df = df.fillna(0.0).astype('float64')
    pd.options.mode.chained_assignment = None

    df["target"][0*subject:1*subject] = "0"
    df["target"][1*subject:2*subject] = "1"
    df["target"][2*subject:3*subject] = "2"

    df["target"][3*subject:4*subject] = "0"
    df["target"][4*subject:5*subject] = "1"
    df["target"][5*subject:6*subject] = "2"

    df["target"][6*subject:7*subject] = "0"
    df["target"][7*subject:8*subject] = "1"
    df["target"][8*subject:9*subject] = "2"

    df["target"][9*subject:10*subject] = "0"
    df["target"][10*subject:11*subject] = "1"
    df["target"][11*subject:12*subject] = "2"
    df.astype('float64')
    df.to_csv('./dataset_eeg_3.csv')
    
    return df
            


In [11]:
dataset_csv_3()

Relative, Abs : Relative
subject : 20


C:\Users\hyg4438\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32,target
0,4.512153,2.236848,16.237565,8.485316,23.868040,8.095887,61.092520,63.790479,68.686679,69.925212,...,8.735276,0.828638,1.010004,1.778969,6.435780,14.978180,11.507832,11.162954,9.491918,0
1,4.539769,4.310226,4.959007,3.965357,7.222414,5.466902,16.751086,19.945360,30.425709,24.128985,...,18.794290,9.741533,7.106003,10.521237,8.559514,18.069939,15.457455,22.764836,22.393518,0
2,9.876811,9.070304,3.572186,4.272411,12.559673,19.156709,26.778865,17.134912,53.187562,49.574373,...,11.408656,6.724140,8.721257,10.193054,2.678986,11.522934,11.118133,11.282202,10.760432,0
3,4.224107,6.190282,8.670873,8.869054,25.241644,20.749929,40.968150,43.190059,42.509827,41.379676,...,10.493111,4.743475,5.994422,2.769252,5.845854,11.149340,12.461344,12.748985,11.845096,0
4,18.441049,19.286110,8.901129,12.888736,7.272771,15.879294,23.182914,38.046045,29.140603,32.524267,...,14.825606,16.428830,15.406232,16.088470,17.454960,18.750681,17.095190,13.764769,13.443134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,9.377716,6.913374,5.995995,4.562073,6.141009,6.275420,17.190073,19.530794,29.798448,27.170256,...,5.781882,11.865520,10.328138,10.171290,11.796915,15.583215,16.539841,15.194092,14.108681,2
236,11.906533,10.057115,11.180124,11.478218,14.621087,16.359638,26.560596,24.297633,30.679741,26.798845,...,16.024479,6.548399,6.806564,10.569691,14.455136,15.406702,16.142647,14.712165,14.551317,2
237,10.335814,8.589677,13.796878,13.048695,25.629270,24.416874,33.204969,34.318945,29.184918,28.485295,...,24.964200,19.061720,18.617480,16.023406,20.075270,26.804320,25.657373,24.098649,19.401579,2
238,14.781336,13.040448,5.557476,4.101700,19.404795,10.489655,27.480708,24.014191,32.643752,30.794355,...,20.821745,9.054677,14.631369,11.264431,17.292341,19.074393,22.937862,23.429094,12.989213,2


In [6]:
eeg_data_3 = dataset_csv_3()

Relative, Abs : Relative
subject : 20


C:\Users\hyg4438\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [7]:


ds_train = {}
ds_test = {}


# train : open_eye + 1-back
ds_train['open_eye_1back']= eeg_data_3.iloc[0:40]
ds_test['open_eye_1back'] = eeg_data_3.iloc[40:60]

# train : open_eye + 2-back
ds_train['open_eye_2back']= eeg_data_3.iloc[60:100]
ds_test['open_eye_2back']= eeg_data_3.iloc[100:120]

# train : open_eye + rest 
ds_train['open_eye_rest']= eeg_data_3.iloc[180:220]
ds_test['open_eye_rest']= eeg_data_3.iloc[220:240]

# train : close_eye + rest
ds_train['close_eye_rest']=eeg_data_3.iloc[120:160]
ds_test['close_eye_rest']=eeg_data_3.iloc[160:180]

# train : total
ds_train['total']=eeg_data_3.iloc[0:40].append( eeg_data_3.iloc[60:100]).append(eeg_data_3.iloc[120:160]).append(eeg_data_3.iloc[180:220])
ds_test['total']= eeg_data_3.iloc[40:60].append( eeg_data_3.iloc[100:120]).append(eeg_data_3.iloc[160:180]).append(eeg_data_3.iloc[220:240])



무작위로 선택된 수천 명의 사람에게 복잡한 질문을 하고 대답을 모으면 많은 경우 전문가의 답보다 이렇게 모은 답이 낫습니다. 

이를 **대중의 지혜(wisdom of crowd)** 라고 합니다.



이와 비슷하게 일련의 예측기로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있습니다. 

일련의 예측기를 **앙상블(ensemble)** 이라고 부릅니다. 


Decision tree의 ensemble을 **Random Forest**라고 합니다. 

---

 * 의사결정나무(Decision Tree)
 
     ![image](https://user-images.githubusercontent.com/39263586/74001911-a6d0e080-49b0-11ea-9fdc-3305bc05f1c2.png)


의사결정나무는 데이터를 분석하여 이들 사이에 존재하는 패턴을 예측 가능한 규칙들의 조합으로 나타내며, 그 모양이 ‘나무’와 같다고 해서 의사결정나무라 불립니다. 질문을 던져서 대상을 좁혀나가는 ‘스무고개’ 놀이와 비슷한 개념입니다. 

---


* Linear Discriminant Analysis (LDA)은 훈련과정에서 클래스 사이를 가장 잘 구분하는 축을 학습합니다. 이 축은 데이터가 투영되는 hyperplane을 정의하는데 사용할 수 있습니다.
이 알고리즘의 장점은 projection을 통해 가능한 한 클래스를 멀리 떨어지게 유지하므로 SVM과 같은 다른 classification 알고리즘을 적용하기 전에 차원을 축소시키는데 좋습니다

RandomForest는 Decision Tree를 1000개를 사용

In [8]:
def M_N_O2 (tasks, k_parameter, num_iter):

    for mode in ['no_lda','lda']:
        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-1])
            Y = np.array(train_3['target'])

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)

            all_scores = []
            Mouse = 0
            Nose = 0
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y):

                    X_final = X
                    lda = LinearDiscriminantAnalysis(n_components=1)
                    if(mode == 'lda') : 
                        lda.fit(X,Y)
                        X_final = lda.transform(X)

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                    test = np.array(test_3.iloc[:,0:-1])
                    if mode == 'lda':
                        test = lda.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa=='0')
                    Nose = Nose+ np.count_nonzero(aaa=='1')

            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))


---

In [9]:

tasks5 = [
     ('close_eye_rest','close_eye_rest'),
     ('close_eye_rest','open_eye_rest'),
     ('close_eye_rest','open_eye_1back'),
     ('close_eye_rest','open_eye_2back'),
     ('close_eye_rest','total')
 ]


#tasks6 = [
#     ('close_eye_rest','open_eye_rest'),
#     ('close_eye_rest','open_eye_1back'),
#     ('close_eye_rest','open_eye_2back'),
# ]

---

close eye rest로 가장 자극이 없는 상태를 training 시킴


In [12]:
M_N_O2(tasks5,k_parameter = 10,num_iter = 10)

-------------------
training:close_eye_rest/test:close_eye_rest/mode:no_lda
model_score:0.1475
mouse:835(0.42), nose:1165(0.58) / total:2000
-------------------
training:close_eye_rest/test:open_eye_rest/mode:no_lda
model_score:0.1775
mouse:676(0.34), nose:1324(0.66) / total:2000
-------------------
training:close_eye_rest/test:open_eye_1back/mode:no_lda
model_score:0.165
mouse:387(0.19), nose:1613(0.81) / total:2000
-------------------
training:close_eye_rest/test:open_eye_2back/mode:no_lda
model_score:0.1675
mouse:517(0.26), nose:1483(0.74) / total:2000
-------------------
training:close_eye_rest/test:total/mode:no_lda
model_score:0.1825
mouse:2449(0.31), nose:5551(0.69) / total:8000
-------------------
training:close_eye_rest/test:close_eye_rest/mode:lda
model_score:0.9425
mouse:1013(0.51), nose:987(0.49) / total:2000
-------------------
training:close_eye_rest/test:open_eye_rest/mode:lda
model_score:0.9525
mouse:1000(0.5), nose:1000(0.5) / total:2000
-------------------
training:cl

---

LDA를 적용하지 않았을 경우 모델 신뢰도가 10~20%로 낮으나
LDA를 적용한 경우 모델 신뢰도가 95% 로 증가함

---


환자 데이터가 20명으로 적기 때문에


10-fold validation으로 교차 검증을 하고


10번 반복으로 신뢰도를 높였다


total :  20 x 10 x 10 = 2000

---

train : close eye rest

* open eye rest : mouse:100(0.5), nose:100(0.5)
* open eye 1 back : mouse:71(0.35), nose:129(0.65)
* open eye 2 back : mouse:51(0.26), nose:149(0.74)

---

작업 복잡도가 증가함에 따라 Mouse O2 breathing을 nose breathing으로 잘 판단하는 것을 확인할 수 있다. 
